In [1]:
# Competitor's Keyword analysis
## Done by Calvin Liu
## info@kicsv.org & calvinliu@stanford.edu

# Similar Companies Facebook Keyword analysis
#### Natural Language Processing

Import needed libraries for Keyword Analysis and Term Frequency-Inverse document frequency

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

import sklearn as sk 
import string
from sklearn.feature_extraction.text import CountVectorizer as CountV
from sklearn.feature_extraction.text import TfidfTransformer as tfidf
from nltk.tokenize import TreebankWordTokenizer

from pprint import pprint

/Users/calvinliu/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/calvinliu/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/calvinliu/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/calvinliu/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/calvinliu/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


importing nltk library and corpus, we definitely could use other corpus however nltk is easy to use for this example purpose.

In [3]:
### We can use stanford CoreNLP
# Using Stanford CoreNLP
# wget http://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip
# unzip stanford-corenlp-full-2017-06-09.zip
# python corenlp.py
# from corenlp import *
# corenlp = StanfordCoreNLP()  # wait a few minutes...
# corenlp.parse("Parse this sentence.")
# from nltk.tokenize.stanford import CoreNLPTokenizer

import nltk 
#(Natural Language toolkit)
# nltk.download() # Download the nltk corpus

from nltk.corpus import stopwords

In [4]:
def make_soup(url, type ="lxml"):
    """
    Make our lives easier with urlopen from urllib2 and BeautifulSoup
    """
    html = urlopen(url).read()
    return BeautifulSoup(html,type)

In [5]:
def remove_html(string):
    """replace html-like characters as empty space"""
    chr = re.compile("<.*?>")
    string = re.sub(chr,"",string)
    return string

In [6]:
def get_posts(section_url):
    """
    Use soup to crawl the webpage and find the user post that 
    was div and _5pbx user content tags (we can press F12 to 
    find the pattern and structure)
    """
    # _5pbx userContent _3576, yields us facebook posts that consists of hyperlink
    soup =make_soup(section_url)
    userContent = soup.find_all("div", "_5pbx userContent _3576")
    # soup.p can change to soup.find_all("div", "_5pbx userContent _3576") something more general use case
    posts = [remove_html(str(s)) for s in userContent]
    return posts
    

We are going to run this analysis on WeWork, YCombinator, StartX, 500 Startups, InnoSpring, and etc. 
(We can extend this list)

In [7]:
BASE_URLs = ["https://www.facebook.com/WeWork/", "https://www.facebook.com/YCombinator/", 
             "https://www.facebook.com/StartX/", "https://www.facebook.com/500startups/",
             "https://www.facebook.com/InnoSpring/", "https://www.facebook.com/amplifyla/",
             "https://www.facebook.com/CcHUBNigeria/","https://www.facebook.com/mattervc/",
             "https://www.facebook.com/UpWestLabsSV","https://www.facebook.com/parisoma/",
             "https://www.facebook.com/SandboxSuites/", "https://www.facebook.com/angelpadorg/",
             "https://www.facebook.com/FoundersSpace/", "https://www.facebook.com/ZGCSiliconValley/"
             ]

In [8]:
posts =[]
# Obtain the post from all competitors

for i in range(len(BASE_URLs)):
    # Go to website i, to get user Contents
    n = get_posts(BASE_URLs[i])

    for post in range(len(n)):
        # Add those individual user content as post.
        posts.append(n[post])
        
# Import stop words for english to remove redundant or meaningless words
stop_words = set(stopwords.words('english'))
stop_words.add(u'around')

# Remove zgc
stop_words.add(u'zgc')


In [9]:
# Bag of word model

def bag(corpus, stop = True, WordOnly = True):
    if stop == True:
        if WordOnly == True:
            vectorizer = CountV(stop_words = stop_words, tokenizer = TreebankWordTokenizer().tokenize)
        else:
            vectorizer = CountV(stop_words = stop_words)
    else:
        if WordOnly == True:
            vectorizer = CountV(stop_words = None, tokenizer = TreebankWordTokenizer().tokenize)
        else:
            vectorizer = CountV(stop_words = None)
    vect = vectorizer.fit_transform(corpus)
    dMat = vect.todense()
    vocab = vectorizer.vocabulary_
    inverted_vocab = dict([[v,k] for k, v in vocab.items()])
    return [vectorizer, dMat, vocab, inverted_vocab]

In [10]:
def frequency(matrix):
    transformer = tfidf(smooth_idf = False)
    mat = transformer.fit_transform(matrix)
    return mat.toarray()

In [11]:
def pretty(l):
    j=[]
    for word in l:
        word= word[1:]
        j.append(word)
    return j

In [12]:
n=100

import math

l = []
r = []
for i in range(n):
    [vectorizer, dMat, vocab, inverted_vocab] = bag(posts, True)
    # use term frequency-inverse document frequency
    # print(dMat)
    tfidf_matrix =frequency(dMat)
    most_used_words_tfidf = np.argmax(tfidf_matrix)
    v = most_used_words_tfidf
    u = inverted_vocab[int(most_used_words_tfidf)%int(len(tfidf_matrix[0]))]
    stop_words.add(u)
    l.append(u)
    r.append(math.ceil(v/100)) 
    # To convert the v, the Tfidf number to smaller magnitude


In [13]:
d = dict(zip(l,r)) # Combine two lists into a dictionary

In [14]:
s_keywords = sorted(d, key=d.get,reverse = True) 
# sort the dictionary(i.e. keywords) with the values.
# print(s_keywords)

In [15]:
s=""
for i in s_keywords:
    s+= i+" / "
    
print(s)

https / //www.foundersspace.com/…/what-are-bitcoin-and-the-b…/ / blockchain / bitcoin / ? / : / @ / wednesedaymotivation / help / get / hump / day / parisoma / # / 's / fab10 / ! / ycombinator / rankings. / share / spot / shares / 10th / top / anniversary / 2017 / happy / accelerator / angelpad / us / crepes / coworking / fund / stampli / week / pymnts.com / series / announcement / coverage / gets / last / following / great / stanbic / ibtc / challenge / stock / think / solution / weecare / customers / challenge. / improve / online / experience / bit.ly/sicc2018 / info / - / innospring / $ / , / apply / zeevi / personalized / microbiome / nutrition / gut / based / david / //blog.ycombinator.com/david-zeevi-on-personalized-n…/ / s18 / ’ / startups / today / http / launched / demo / 59 / wework / combinator / summer / //ow.ly/gpep30lvmgl / 2 / positive / camp / energy / experiencing / . / wwcamp / together / year / global / let / every / gather / begin / camp. / community / ... / adventu

As you can see from the list s, our competitors usually uses positive words for the public while they always put links and hashtag on their post to direct and attract traffic to their website/events. Moreover, they use words like "today","hump day", "2017" and "happy" to interact with audience ocassionally. And more formal interaction with "announcement" and "Apply."

In addition, they also used buzzwords for startups, like "global", "stock" "$", and "fund." (and more~). They also tagged others with '@'.

Later stage we can further analyze the list of keywords. Right now, we are only looking at 100 words. When we have more competitors to search OR when we havve more 'posts' saved in our DB, we can further analyze the general strategy and other competitors. 